## Imports

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Loading Data

In [4]:
df_train = pd.read_csv('./data/data_train.csv')
df_train.head()

df_test = pd.read_csv('./data/data_validation.csv')
df_test.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
2,769,1,2.9,1,0,0,9,0.1,182,5,...,248,874,3946,5,2,7,0,0,0,3
3,775,0,1.0,0,3,0,46,0.7,159,2,...,862,1864,568,17,15,11,1,1,1,0
4,595,0,0.9,1,7,1,23,0.1,121,3,...,441,810,3752,10,2,18,1,1,0,3


In [5]:
X_train = df_train.drop('price_range',axis=1).values
y_train = df_train['price_range'].values

X_test = df_test.drop('price_range',axis=1).values
y_test = df_test['price_range'].values

## KNN From Scratch

In [ ]:
class KNeighborsClassifier():
    def __init__(self, n_neighbors, metric):
        self.n_neighbors = n_neighbors
        self.metric = metric
        self.X_train_scaled = []
        self.y_train = []
    def fit(self, X_train_scaled,y_train): 
        """
        :param X_train_scaled: 
        :param y_train: 
        :return: 
        """
        self.X_train_scaled = X_train_scaled
        self.y_train = y_train
    # def predict(self):
        
        